### Pytorch Model 1

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

### 모델 정의(CNN)

In [6]:
class CustomCNN(nn.Module):
    def __init__(self,
                 input_shape=(3, 64, 64), # 입력 이미지 크기(채널, 높이, 너비)
                 num_classes=10, # 출력 클래스 수
                 depth=2, # Conv 블록 개수 (예: 2->경량, 4->고성능(더 깊어짐))
                 base_channels=32, # 첫 Conv 블록의 출력 채널 수
                 activation='relu', # 활성화 함수 선택(relu, gelu, leaky_relu, sigmoid, 등)
                 kernel_size=3, # Conv 필터 크기
                 dropout_rate=0.25, # Conv 블록 내 Dropout 비율
                 padding='same', # 패딩 모드 : same, valid, 정수 등
                 use_batchnorm=False # 배치 정규화 사용 여부
                 ):
        super(CustomCNN, self).__init__()
        
        # 활성화 함수 매핑 (클래스 기반)
        activation_map = {
            'relu' : nn.ReLU,
            'leaky_relu' : nn.LeakyReLU,
            'gelu' : nn.GELU,
            'tanh' : nn.Tanh,
            'sigmoid' : nn.Sigmoid
        }
        
        if activation not in activation_map:
            raise ValueError(f"Unsupported activation : {activation}. Choose from {list(activation_map.keys())}")
        self.activation_cls = activation_map[activation]
        
        self.pool = nn.MaxPool2d(2, 2) #
        
        layers = []
        in_channels = input_shape[0]
        
        for i in range(depth):
            out_channels = base_channels * (2**i)
            #
            pad = self.resolve_padding(padding, kernel_size)
            
            # 첫 번째 Conv block
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=pad))
            if use_batchnorm:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(self.activation_cls())
            
            # 두 번째 ~ Conv block
            if i < depth -1 :
                layers.append(nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, padding=pad))
                if use_batchnorm:
                    layers.append(nn.BatchNorm2d(out_channels))
                layers.append(self.activation_cls())
                
            # 풀링 + 드롭아웃
            layers.append(self.pool)
            layers.append(nn.Dropout(dropout_rate))
            
            in_channels = out_channels
            
        self.conv_block = nn.Sequential(*layers)
                
        # Flatten 크기 자동 계산
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            x = self.conv_block(dummy)
            self.flatten_dim = x.view(1, -1).shape[1]
            
        # Fully-connected Layer
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flatten_dim, 128),
            self.activation_cls(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
        
    # 패딩 계산 함수
    @staticmethod
    def resolve_padding(padding, kernel_size):
        if isinstance(padding, str):
            if padding.lower() == 'same':
                if isinstance(kernel_size, int):
                    return kernel_size // 2
                elif isinstance(kernel_size, tuple):
                    return tuple(k // 2 for k in kernel_size)
            elif padding.lower() == 'valid':
                return 0
            else:
                raise ValueError(f"Unsupported padding String : {padding}")
        elif isinstance(padding, int):
            return padding
        elif isinstance(padding, tuple):
            return padding
        else:
            raise ValueError(f"Unsupported padding type : {type(padding)}")
    # /패딩 계산 함수
        
    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc
        _layers(x)
        return x

### 학습 함수

In [9]:
def train_one_epoch(model, dataloader, loss_fn, optimizer, device):
    model.train() # 모델을 학습 모드로 전환
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 1. 옵티마이저 초기화
        optimizer.zero_grad()
        
        # 2. 순전파
        outputs = model(inputs)
        
        # 3. 손실 계산
        loss = loss_fn(outputs, labels)
        
        # 4. 역전파
        loss.backward()
        
        # 5. 파라키터 업데이트
        optimizer.step()
        
        # 통계 기록
        running_loss += loss.item() * inputs.size(0)
        
        # 에측 결과
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    print(f"Train Loss : {epoch_loss:.4f}, Accuracy : {epoch_acc:.4f}")
    return epoch_loss, epoch_acc

### 전체 학습 루프 함수

In [ ]:
def train_model(model, trainloader, loss_fn, optimizer, device, epochs=10):
    

### 예시

In [8]:
# 모델 , 손실 함수, 옵티마이저 정의 

# 1. 모델 정의
model = CustomCNN(
    input_shape=(3, 64, 64),
    num_classes=10,
    depth=3,
    base_channels=32,
    activation='relu',
    kernel_size=3,
    dropout_rate=0.25,
    padding='same',
    use_batchnorm=True
)

# 2. GPU 사용 가능하면 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

# 3. 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# 4. 옵티마이저 정의
optimizer = optim.Adam(model.parameters(), lr=0.001)

cuda


In [ ]:
# 